# Concrete Autoencoders dMRI for PyTorch

In [1]:
import project_path  # Always import this first

In [2]:
from pathlib import Path

import numpy as np
import torch

from autoencoder.env import DATA_PATH
from autoencoder.logger import logger, logging_tqdm

In [3]:
ROOT_PATH = Path().cwd().parent

In [4]:
logger.info("torch version %s", torch.__version__)

2021-07-18 20:28:51,418 - geometric-dl - INFO - torch version 1.9.0 (<ipython-input-4-3c2fef3d866b>:1)


In [5]:
# use gpu if available, else cpu
has_cuda = torch.cuda.is_available()

logger.info("Is the GPU available? %s", has_cuda)
logger.info("Current device: %s", torch.cuda.current_device())
logger.info("Device count: %s", torch.cuda.device_count())

device = torch.device("cuda" if has_cuda else "cpu")
if has_cuda:
    logger.info("Using device: %s", torch.cuda.get_device_properties(device))
else:
    logger.warning("No GPU dectected! Training will be extremly slow")

2021-07-18 20:28:51,485 - geometric-dl - INFO - Is the GPU available? True (<ipython-input-5-a5029b49c980>:4)
2021-07-18 20:28:51,489 - geometric-dl - INFO - Current device: 0 (<ipython-input-5-a5029b49c980>:5)
2021-07-18 20:28:51,489 - geometric-dl - INFO - Device count: 1 (<ipython-input-5-a5029b49c980>:6)
2021-07-18 20:28:51,490 - geometric-dl - INFO - Using device: _CudaDeviceProperties(name='NVIDIA GeForce GTX 1080', major=6, minor=1, total_memory=8118MB, multi_processor_count=20) (<ipython-input-5-a5029b49c980>:10)


## Experiments

In [6]:
import pickle as pk
from datetime import datetime

from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from autoencoder.concrete import (
    ConcreteAutoencoderFeatureSelector,
    decoder_1l,
    decoder_2l,
    decoder_3l,
)
from autoencoder.dataset import MRISelectorSubjDataset

In [7]:
# import modules to build RunBuilder and RunManager helper classes
from collections import OrderedDict
from collections import namedtuple
from itertools import product

# Read in the hyper-parameters and return a Run namedtuple containing all the
# combinations of hyper-parameters
class RunBuilder:
    @staticmethod
    def get_runs(params):

        Run = namedtuple("Run", params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

In [8]:
def train_model(train_subject, test_subject, params):
    """
    Trains the ConcreteAutoencoderFeatureSelector

    Parameters:
        train_subject (List): subjects to train on
        test_subject (List): subjects to test on
        params (Dict): model parameters to grid search on.
    """
    strftime = "%Y%m%d%H%M%S"
    writer = SummaryWriter(
        log_dir=Path(ROOT_PATH, "runs", datetime.now().strftime(strftime))
    )

    torch.manual_seed(14)

    for run in RunBuilder.get_runs(params):
        logger.info("running: %s", run)

        del_feats = [1148, 912]

        now = datetime.now()
        logger.info(run.exclude)
        exclude_str = "-".join(map(str, run.exclude))
        model_info_template_str = f"{now:%Y%m%d%H%M%S}_lr={run.lr}_batch_size={run.batch_size}_num_epochs={run.num_epochs}_n_features={run.n_features}_decoder={run.decoder.__name__}_test={test_subject[0]}_exclude={exclude_str}"

        checkpoint_path = str(
            Path(ROOT_PATH, "runs", "models", f"{model_info_template_str}_runtime.h5")
        )
        monitor_callback = ModelCheckpoint(
            checkpoint_path, monitor="val_loss", verbose=True
        )

        root_dir = Path(ROOT_PATH, "data")
        dataf = "data_.hdf5"
        headerf = "header_.csv"
        subj_list_train = np.array(train_subject)
        subj_list_valid = np.array(test_subject)

        train_set = MRISelectorSubjDataset(
            root_dir, dataf, headerf, subj_list_train, run.exclude
        )

        train_gen = DataLoader(
            train_set,
            batch_size=run.batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True,
            drop_last=True,
        )

        # for the validation dataset
        valid_set = MRISelectorSubjDataset(root_dir, dataf, headerf, subj_list_valid)
        valid_gen = DataLoader(
            valid_set,
            batch_size=run.batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True,
            drop_last=True,
        )

        # 1st time
        checkpt = False
        # Continue training
        # checkpt = True
        # temp = Tensor([10]) # check last value if necessary

        selector = ConcreteAutoencoderFeatureSelector(
            K=run.n_features,
            decoder=run.decoder,
            device=device,
            num_features=run.n_features,
            num_epochs=run.num_epochs,
            learning_rate=run.lr,
            start_temp=10,
            min_temp=0.1,
            tryout_limit=1,
            input_dim=1344 - len(run.exclude),
            checkpt=checkpt,
            callback=monitor_callback,
            writer=writer,
            path=ROOT_PATH,
        )  # ,losstrain=losstrain,lossval=lossval)

        selector.fit(X=train_gen, val_X=valid_gen)

        model = selector.get_params()
        torch.save(
            model.state_dict(),
            Path(ROOT_PATH, "runs", "models", f"{model_info_template_str}_params.pt"),
        )

        indices = selector.get_indices().to("cpu")
        logger.info(np.sort(indices))
        np.savetxt(
            Path(ROOT_PATH, "runs", "models", f"{model_info_template_str}.txt"),
            np.array(indices, dtype=int),
            fmt="%d",
        )

### Model training

We use a learning rate of 0.001, batch size of 265 and 2000 epochs. 2000 Epochs is likely not enough to get a high mean max of probabilities, but otherwise training takes too long. Our input size is 1344, so for the latent space we take half that, and continue halving for five more latent space sizes. Lastly we have 3 decoders of various complexities. `decoder_1l` being the least complex and `decoder_3l` the most complex.

In [9]:
train_model(
    [11, 12, 13, 14],
    [15],
    OrderedDict(
        lr=[0.001],
        batch_size=[256],
        num_epochs=[2000],
        n_features=[21, 42, 84, 168, 336, 672],  # latent space sizes
        decoder=[decoder_1l, decoder_2l, decoder_3l],
        exclude=[[]],  # features to exlude from training
    ),
)

2021-07-08 14:46:51,739 - geometric-dl - INFO - running: Run(lr=0.001, batch_size=256, num_epochs=2000, n_features=21, decoder=<function decoder_1l at 0x7fcd6540c670>, exclude=[]) (<ipython-input-8-85c3dc78dc0b>:18)
2021-07-08 14:46:51,741 - geometric-dl - INFO - [] (<ipython-input-8-85c3dc78dc0b>:23)
2021-07-08 14:46:59,416 - geometric-dl - INFO - steps per epoch: 1830 (feature_selector.py:62)
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
torch.Size([256, 1344])
to

KeyboardInterrupt: 

In [9]:
train_model(
    [11, 12, 13, 14],
    [15],
    OrderedDict(
        lr=[0.001],
        batch_size=[256],
        num_epochs=[2000],
        n_features=[336],  # latent space sizes
        decoder=[decoder_3l],
        exclude=[[1148], [912, 1148]],
    ),
)

2021-07-18 20:28:57,302 - geometric-dl - INFO - running: Run(lr=0.001, batch_size=256, num_epochs=2000, n_features=336, decoder=<function decoder_3l at 0x7f8cc554a9d0>, exclude=[1148]) (<ipython-input-8-85c3dc78dc0b>:18)
2021-07-18 20:28:57,304 - geometric-dl - INFO - [1148] (<ipython-input-8-85c3dc78dc0b>:23)
2021-07-18 20:29:05,811 - geometric-dl - INFO - steps per epoch: 1830 (feature_selector.py:62)


/home/maarten/miniconda3/envs/mudi/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([256, 1344])) that is different to the input size (torch.Size([256, 1343])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (1343) must match the size of tensor b (1344) at non-singleton dimension 1